In [3]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [5]:
bureau = pd.read_csv('../input/bureau.csv', nrows = None)
bb = pd.read_csv('../input/bureau_balance.csv', nrows = None)

In [6]:
bb, bb_cat = one_hot_encoder(bb, True)

In [7]:
bb

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,STATUS_nan
0,5715448,0,0,0,0,0,0,0,1,0,0
1,5715448,-1,0,0,0,0,0,0,1,0,0
2,5715448,-2,0,0,0,0,0,0,1,0,0
3,5715448,-3,0,0,0,0,0,0,1,0,0
4,5715448,-4,0,0,0,0,0,0,1,0,0
5,5715448,-5,0,0,0,0,0,0,1,0,0
6,5715448,-6,0,0,0,0,0,0,1,0,0
7,5715448,-7,0,0,0,0,0,0,1,0,0
8,5715448,-8,0,0,0,0,0,0,1,0,0
9,5715448,-9,1,0,0,0,0,0,0,0,0


In [8]:
bb_cat


['STATUS_0',
 'STATUS_1',
 'STATUS_2',
 'STATUS_3',
 'STATUS_4',
 'STATUS_5',
 'STATUS_C',
 'STATUS_X',
 'STATUS_nan']

In [9]:
#original_columns = list(bureau.columns)

In [10]:
#original_columns

In [11]:
#categorical_columns = [col for col in bureau.columns if bureau[col].dtype == 'object']

In [12]:
#categorical_columns

In [13]:
#bureau = pd.get_dummies(bureau, columns= categorical_columns, dummy_na= True)

In [14]:
#bureau

In [15]:
#bureau_cat = [c for c in bureau.columns if c not in original_columns]

In [16]:
#bureau_cat

In [17]:
bureau, bureau_cat = one_hot_encoder(bureau, True)

In [18]:
bureau_cat

['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency 1',
 'CREDIT_CURRENCY_currency 2',
 'CREDIT_CURRENCY_currency 3',
 'CREDIT_CURRENCY_currency 4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another type of loan',
 'CREDIT_TYPE_Car loan',
 'CREDIT_TYPE_Cash loan (non-earmarked)',
 'CREDIT_TYPE_Consumer credit',
 'CREDIT_TYPE_Credit card',
 'CREDIT_TYPE_Interbank credit',
 'CREDIT_TYPE_Loan for business development',
 'CREDIT_TYPE_Loan for purchase of shares (margin lending)',
 'CREDIT_TYPE_Loan for the purchase of equipment',
 'CREDIT_TYPE_Loan for working capital replenishment',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mobile operator loan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_Real estate loan',
 'CREDIT_TYPE_Unknown type of loan',
 'CREDIT_TYPE_nan']

In [19]:
bureau

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_TYPE_nan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.000,0.000,...,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.000,171342.000,...,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.500,NaN,...,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.000,NaN,...,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.500,0,2700000.000,NaN,...,0,0,0,0,0,0,0,0,0,0
5,215354,5714467,-273,0,27460.0,NaN,0.000,0,180000.000,71017.380,...,0,0,0,0,0,0,0,0,0,0
6,215354,5714468,-43,0,79.0,NaN,0.000,0,42103.800,42103.800,...,0,0,0,0,0,0,0,0,0,0
7,162297,5714469,-1896,0,-1684.0,-1710.0,14985.000,0,76878.450,0.000,...,0,0,0,0,0,0,0,0,0,0
8,162297,5714470,-1146,0,-811.0,-840.0,0.000,0,103007.700,0.000,...,0,0,0,0,0,0,0,0,0,0
9,162297,5714471,-1146,0,-484.0,NaN,0.000,0,4500.000,0.000,...,0,0,0,0,0,0,0,0,0,0


In [20]:
bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}

In [21]:
bb_aggregations

{'MONTHS_BALANCE': ['min', 'max', 'size']}

In [22]:
for col in bb_cat:
        bb_aggregations[col] = ['mean']

In [23]:
bb_aggregations

{'MONTHS_BALANCE': ['min', 'max', 'size'],
 'STATUS_0': ['mean'],
 'STATUS_1': ['mean'],
 'STATUS_2': ['mean'],
 'STATUS_3': ['mean'],
 'STATUS_4': ['mean'],
 'STATUS_5': ['mean'],
 'STATUS_C': ['mean'],
 'STATUS_X': ['mean'],
 'STATUS_nan': ['mean']}

In [24]:
bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)

In [25]:
bb_agg

MONTHS_BALANCE           STATUS_0  STATUS_1 STATUS_2 STATUS_3  \
                        min max size      mean      mean     mean     mean   
SK_ID_BUREAU                                                                 
5001709                 -96   0   97  0.000000  0.000000      0.0      0.0   
5001710                 -82   0   83  0.060241  0.000000      0.0      0.0   
5001711                  -3   0    4  0.750000  0.000000      0.0      0.0   
5001712                 -18   0   19  0.526316  0.000000      0.0      0.0   
5001713                 -21   0   22  0.000000  0.000000      0.0      0.0   
5001714                 -14   0   15  0.000000  0.000000      0.0      0.0   
5001715                 -59   0   60  0.000000  0.000000      0.0      0.0   
5001716                 -85   0   86  0.313953  0.000000      0.0      0.0   
5001717                 -21   0   22  0.772727  0.000000      0.0      0.0   
5001718                 -38   0   39  0.615385  0.051282      0.0      0.0   
5001719                 -42   0   43  0.046512  0.000000      0.0      0.0   
5001720                 -35   0   36  0.805556  0.194444      0.0      0.0   
5001721                 -84   0   85  0.152941  0.000000      0.0      0.0   
5001722                 -85   0   86  0.313953  0.232558      0.0      0.0   
5001723                 -30   0   31  0.258065  0.000000      0.0      0.0   
5001724                 -30   0   31  0.000000  0.000000      0.0      0.0   
5001725                  -7   0    8  0.875000  0.000000      0.0      0.0   
5001726                 -38   0   39  0.153846  0.000000      0.0      0.0   
5001727                 -94   0   95  0.242105  0.000000      0.0      0.0   
5001728                   0   0    1  1.000000  0.000000      0.0      0.0   
5001729                  -6   0    7  0.857143  0.000000      0.0      0.0   
5001730                 -60   0   61  0.081967  0.000000      0.0      0.0   
5001731                 -10   0   11  0.000000  0.000000      0.0      0.0   
5001732                 -35   0   36  0.000000  0.000000      0.0      0.0   
5001733                 -10   0   11  0.000000  0.000000      0.0      0.0   
5001734                  -2   0    3  1.000000  0.000000      0.0      0.0   
5001735                 -23   0   24  0.041667  0.000000      0.0      0.0   
5001736                 -23   0   24  0.166667  0.000000      0.0      0.0   
5001737                 -21   0   22  0.545455  0.000000      0.0      0.0   
5001738                 -43   0   44  0.272727  0.000000      0.0      0.0   
...                     ...  ..  ...       ...       ...      ...      ...   
6842859                 -35   0   36  0.166667  0.000000      0.0      0.0   
6842860                 -35   0   36  0.805556  0.000000      0.0      0.0   
6842861                 -35   0   36  0.638889  0.000000      0.0      0.0   
6842862                 -68   0   69  0.246377  0.000000      0.0      0.0   
6842863                 -78   0   79  0.012658  0.000000      0.0      0.0   
6842864                 -34   0   35  0.457143  0.000000      0.0      0.0   
6842865                 -18   0   19  1.000000  0.000000      0.0      0.0   
6842866                  -4   0    5  1.000000  0.000000      0.0      0.0   
6842867                 -14   0   15  1.000000  0.000000      0.0      0.0   
6842868                 -55   0   56  0.071429  0.000000      0.0      0.0   
6842869                 -26   0   27  0.074074  0.000000      0.0      0.0   
6842870                 -35   0   36  0.111111  0.000000      0.0      0.0   
6842871                 -18   0   19  0.631579  0.000000      0.0      0.0   
6842872                 -87   0   88  0.079545  0.000000      0.0      0.0   
6842873                 -55   0   56  0.071429  0.000000      0.0      0.0   
6842874                 -89   0   90  0.088889  0.011111      0.0      0.0   
6842875                 -76   0   77  0.311688  0.000000      0.0      0.0   
6842876                 -25   0   2

In [26]:
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

In [27]:
bb_agg.columns

Index(['MONTHS_BALANCE_MIN', 'MONTHS_BALANCE_MAX', 'MONTHS_BALANCE_SIZE',
       'STATUS_0_MEAN', 'STATUS_1_MEAN', 'STATUS_2_MEAN', 'STATUS_3_MEAN',
       'STATUS_4_MEAN', 'STATUS_5_MEAN', 'STATUS_C_MEAN', 'STATUS_X_MEAN',
       'STATUS_nan_MEAN'],
      dtype='object')

In [28]:
bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')

In [29]:
list(bureau)

['SK_ID_CURR',
 'SK_ID_BUREAU',
 'DAYS_CREDIT',
 'CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE',
 'CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'DAYS_CREDIT_UPDATE',
 'AMT_ANNUITY',
 'CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency 1',
 'CREDIT_CURRENCY_currency 2',
 'CREDIT_CURRENCY_currency 3',
 'CREDIT_CURRENCY_currency 4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another type of loan',
 'CREDIT_TYPE_Car loan',
 'CREDIT_TYPE_Cash loan (non-earmarked)',
 'CREDIT_TYPE_Consumer credit',
 'CREDIT_TYPE_Credit card',
 'CREDIT_TYPE_Interbank credit',
 'CREDIT_TYPE_Loan for business development',
 'CREDIT_TYPE_Loan for purchase of shares (margin lending)',
 'CREDIT_TYPE_Loan for the purchase of equipment',
 'CREDIT_TYPE_Loan for working capital replenishment',
 'CREDIT_TYPE_Micro

In [30]:
len(list(bureau))

52

In [31]:
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)

In [32]:
bureau.head(10)

,SK_ID_CURR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,...,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_1_MEAN,STATUS_2_MEAN,STATUS_3_MEAN,STATUS_4_MEAN,STATUS_5_MEAN,STATUS_C_MEAN,STATUS_X_MEAN,STATUS_nan_MEAN
0,215354,-497,0,-153.0,-153.0,NaN,0,91323.000,0.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,-208,0,1075.0,NaN,NaN,0,225000.000,171342.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,-203,0,528.0,NaN,NaN,0,464323.500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,-203,0,NaN,NaN,NaN,0,90000.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,-629,0,1197.0,NaN,77674.500,0,2700000.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,215354,-273,0,27460.0,NaN,0.000,0,180000.000,71017.380,108982.620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,215354,-43,0,79.0,NaN,0.000,0,42103.800,42103.800,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,162297,-1896,0,-1684.0,-1710.0,14985.000,0,76878.450,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,162297,-1146,0,-811.0,-840.0,0.000,0,103007.700,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,162297,-1146,0,-484.0,NaN,0.000,0,4500.000,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
 del bb, bb_agg

In [34]:
gc.collect()

28

 ## Bureau and bureau_balance numeric features

In [ ]:
 num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }

### Bureau and bureau_balance categorical features

In [94]:

cat_aggregations = {}
for cat in bureau_cat: cat_aggregations[cat] = ['mean']


In [101]:
cat_aggregations

{'CREDIT_ACTIVE_Active': ['mean'],
 'CREDIT_ACTIVE_Bad debt': ['mean'],
 'CREDIT_ACTIVE_Closed': ['mean'],
 'CREDIT_ACTIVE_Sold': ['mean'],
 'CREDIT_ACTIVE_nan': ['mean'],
 'CREDIT_CURRENCY_currency 1': ['mean'],
 'CREDIT_CURRENCY_currency 2': ['mean'],
 'CREDIT_CURRENCY_currency 3': ['mean'],
 'CREDIT_CURRENCY_currency 4': ['mean'],
 'CREDIT_CURRENCY_nan': ['mean'],
 'CREDIT_TYPE_Another type of loan': ['mean'],
 'CREDIT_TYPE_Car loan': ['mean'],
 'CREDIT_TYPE_Cash loan (non-earmarked)': ['mean'],
 'CREDIT_TYPE_Consumer credit': ['mean'],
 'CREDIT_TYPE_Credit card': ['mean'],
 'CREDIT_TYPE_Interbank credit': ['mean'],
 'CREDIT_TYPE_Loan for business development': ['mean'],
 'CREDIT_TYPE_Loan for purchase of shares (margin lending)': ['mean'],
 'CREDIT_TYPE_Loan for the purchase of equipment': ['mean'],
 'CREDIT_TYPE_Loan for working capital replenishment': ['mean'],
 'CREDIT_TYPE_Microloan': ['mean'],
 'CREDIT_TYPE_Mobile operator loan': ['mean'],
 'CREDIT_TYPE_Mortgage': ['mean'],
 '

In [ ]:
len(cat_aggregations)

In [102]:
for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']

In [103]:
cat_aggregations

{'CREDIT_ACTIVE_Active': ['mean'],
 'CREDIT_ACTIVE_Bad debt': ['mean'],
 'CREDIT_ACTIVE_Closed': ['mean'],
 'CREDIT_ACTIVE_Sold': ['mean'],
 'CREDIT_ACTIVE_nan': ['mean'],
 'CREDIT_CURRENCY_currency 1': ['mean'],
 'CREDIT_CURRENCY_currency 2': ['mean'],
 'CREDIT_CURRENCY_currency 3': ['mean'],
 'CREDIT_CURRENCY_currency 4': ['mean'],
 'CREDIT_CURRENCY_nan': ['mean'],
 'CREDIT_TYPE_Another type of loan': ['mean'],
 'CREDIT_TYPE_Car loan': ['mean'],
 'CREDIT_TYPE_Cash loan (non-earmarked)': ['mean'],
 'CREDIT_TYPE_Consumer credit': ['mean'],
 'CREDIT_TYPE_Credit card': ['mean'],
 'CREDIT_TYPE_Interbank credit': ['mean'],
 'CREDIT_TYPE_Loan for business development': ['mean'],
 'CREDIT_TYPE_Loan for purchase of shares (margin lending)': ['mean'],
 'CREDIT_TYPE_Loan for the purchase of equipment': ['mean'],
 'CREDIT_TYPE_Loan for working capital replenishment': ['mean'],
 'CREDIT_TYPE_Microloan': ['mean'],
 'CREDIT_TYPE_Mobile operator loan': ['mean'],
 'CREDIT_TYPE_Mortgage': ['mean'],
 '

In [104]:
len(cat_aggregations)

35

In [105]:
bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})

In [107]:
bureau_agg.head(5)

DAYS_CREDIT                                  DAYS_CREDIT_ENDDATE  \
                   min  max         mean            var                 min   
SK_ID_CURR                                                                    
100001           -1572  -49  -735.000000  240043.666667             -1329.0   
100002           -1437 -103  -874.000000  186150.000000             -1072.0   
100003           -2586 -606 -1400.750000  827783.583333             -2434.0   
100004           -1326 -408  -867.000000  421362.000000              -595.0   
100005            -373  -62  -190.666667   26340.333333              -128.0   

                               DAYS_CREDIT_UPDATE CREDIT_DAY_OVERDUE       \
               max        mean               mean                max mean   
SK_ID_CURR                                                                  
100001      1778.0   82.428571         -93.142857                  0  0.0   
100002       780.0 -349.000000        -499.875000                  0  0.0   
100003      1216.0 -544.500000        -816.000000                  0  0.0   
100004      -382.0 -488.500000        -532.000000                  0  0.0   
100005      1324.0  439.333333         -54.333333                  0  0.0   

                 ...       CREDIT_TYPE_nan STATUS_0_MEAN STATUS_1_MEAN  \
                 ...                  mean          mean          mean   
SK_ID_CURR       ...                                                     
100001           ...                     0      0.336651      0.007519   
100002           ...                     0      0.406960      0.255682   
100003           ...                     0           NaN           NaN   
100004           ...                     0           NaN           NaN   
100005           ...                     0      0.735043      0.000000   

           STATUS_2_MEAN STATUS_3_MEAN STATUS_4_MEAN STATUS_5_MEAN  \
                    mean          mean          mean          mean   
SK_ID_CURR                                                           
100001               0.0           0.0           0.0           0.0   
100002               0.0           0.0           0.0           0.0   
100003               NaN           NaN           NaN           NaN   
100004               NaN           NaN           NaN           NaN   
100005               0.0           0.0           0.0           0.0   

           STATUS_C_MEAN STATUS_X_MEAN STATUS_nan_MEAN  
                    mean          mean            mean  
SK_ID_CURR                                              
100001          0.441240      0.214590             0.0  
100002          0.175426      0.161932             0.0  
100003               NaN           NaN             NaN  
100004               NaN           NaN             NaN  
100005          0.128205      0.136752             0.0  

[5 rows x 62 columns]

In [108]:
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

In [109]:
bureau_agg.columns

Index(['BURO_DAYS_CREDIT_MIN', 'BURO_DAYS_CREDIT_MAX', 'BURO_DAYS_CREDIT_MEAN',
       'BURO_DAYS_CREDIT_VAR', 'BURO_DAYS_CREDIT_ENDDATE_MIN',
       'BURO_DAYS_CREDIT_ENDDATE_MAX', 'BURO_DAYS_CREDIT_ENDDATE_MEAN',
       'BURO_DAYS_CREDIT_UPDATE_MEAN', 'BURO_CREDIT_DAY_OVERDUE_MAX',
       'BURO_CREDIT_DAY_OVERDUE_MEAN', 'BURO_AMT_CREDIT_MAX_OVERDUE_MEAN',
       'BURO_AMT_CREDIT_SUM_MAX', 'BURO_AMT_CREDIT_SUM_MEAN',
       'BURO_AMT_CREDIT_SUM_SUM', 'BURO_AMT_CREDIT_SUM_DEBT_MAX',
       'BURO_AMT_CREDIT_SUM_DEBT_MEAN', 'BURO_AMT_CREDIT_SUM_DEBT_SUM',
       'BURO_AMT_CREDIT_SUM_OVERDUE_MEAN', 'BURO_AMT_CREDIT_SUM_LIMIT_MEAN',
       'BURO_AMT_CREDIT_SUM_LIMIT_SUM', 'BURO_AMT_ANNUITY_MAX',
       'BURO_AMT_ANNUITY_MEAN', 'BURO_CNT_CREDIT_PROLONG_SUM',
       'BURO_MONTHS_BALANCE_MIN_MIN', 'BURO_MONTHS_BALANCE_MAX_MAX',
       'BURO_MONTHS_BALANCE_SIZE_MEAN', 'BURO_MONTHS_BALANCE_SIZE_SUM',
       'BURO_CREDIT_ACTIVE_Active_MEAN', 'BURO_CREDIT_ACTIVE_Bad debt_MEAN',
       'BURO_CREDI

###  Bureau: Active credits - using only numerical aggregations

# Preprocess application


In [38]:
df = pd.read_csv('../input/application_train.csv', nrows= None)
test_df = pd.read_csv('../input/application_test.csv', nrows= None)

In [39]:
print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))

Train samples: 307511, test samples: 48744


### Optional: Remove 4 applications with XNA CODE_GENDER (train set)

In [40]:
df = df[df['CODE_GENDER'] != 'XNA']

In [41]:
docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]

In [42]:
docs

['FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [43]:
live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]

In [44]:
live

['FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL']

### NaN values for DAYS_EMPLOYED: 365.243 -> nan

In [46]:
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)

In [49]:
df['DAYS_EMPLOYED'].head(15)

0     -637.0
1    -1188.0
2     -225.0
3    -3039.0
4    -3038.0
5    -1588.0
6    -3130.0
7     -449.0
8        NaN
9    -2019.0
10    -679.0
11       NaN
12   -2717.0
13   -3028.0
14    -203.0
Name: DAYS_EMPLOYED, dtype: float64

In [50]:
 inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

In [51]:
inc_by_org

ORGANIZATION_TYPE
Advertising               165600.0
Agriculture               126000.0
Bank                      157500.0
Business Entity Type 1    157500.0
Business Entity Type 2    157500.0
Business Entity Type 3    157500.0
Cleaning                  135000.0
Construction              180000.0
Culture                   157500.0
Electricity               157500.0
Emergency                 162000.0
Government                135000.0
Hotel                     135000.0
Housing                   135000.0
Industry: type 1          157500.0
Industry: type 10         180000.0
Industry: type 11         137227.5
Industry: type 12         157500.0
Industry: type 13         135000.0
Industry: type 2          157500.0
Industry: type 3          135000.0
Industry: type 4          157500.0
Industry: type 5          157500.0
Industry: type 6          139050.0
Industry: type 7          144000.0
Industry: type 8          162450.0
Industry: type 9          180000.0
Insurance                 157500.0
Ki

In [52]:
df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
df[['NEW_CREDIT_TO_ANNUITY_RATIO']].head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NEW_CREDIT_TO_ANNUITY_RATIO
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.5,24700.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,16.461104
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.5,35698.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,36.234085
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.0,6750.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,20.000000
3,100006,0,Cash loans,F,N,Y,0,135000.000,312682.5,29686.5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,10.532818
4,100007,0,Cash loans,M,N,Y,0,121500.000,513000.0,21865.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,23.461618
5,100008,0,Cash loans,M,N,Y,0,99000.000,490495.5,27517.5,...,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,17.824857
6,100009,0,Cash loans,F,Y,Y,1,171000.000,1560726.0,41301.0,...,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0,37.789061
7,100010,0,Cash loans,M,Y,Y,0,360000.000,1530000.0,42075.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,36.363636
8,100011,0,Cash loans,F,N,Y,0,112500.000,1019610.0,33826.5,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,30.142344
9,100012,0,Revolving loans,M,N,Y,0,135000.000,405000.0,20250.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,20.000000


In [53]:
df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
df[['NEW_CREDIT_TO_GOODS_RATIO']].head(10)

0    1.158397
1    1.145199
2    1.000000
3    1.052803
4    1.000000
5    1.079198
6    1.118800
7    1.000000
8    1.116158
9    1.000000
Name: NEW_CREDIT_TO_GOODS_RATIO, dtype: float64

In [59]:
df['NEW_DOC_IND_AVG'] = df[docs].mean(axis=1)
df[['NEW_DOC_IND_AVG']].head(10)

,NEW_DOC_IND_AVG
0,0.05
1,0.05
2,0.00
3,0.05
4,0.05
5,0.05
6,0.10
7,0.05
8,0.05
9,0.00


In [60]:
df['NEW_DOC_IND_STD'] = df[docs].std(axis=1)
df[['NEW_DOC_IND_STD']].head(10)

,NEW_DOC_IND_STD
0,0.223607
1,0.223607
2,0.000000
3,0.223607
4,0.223607
5,0.223607
6,0.307794
7,0.223607
8,0.223607
9,0.000000


In [62]:
df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
df[['NEW_LIVE_IND_SUM']].head(10)

,NEW_LIVE_IND_SUM
0,4
1,4
2,5
3,3
4,3
5,5
6,4
7,4
8,2
9,3


In [61]:
df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)
df[['NEW_DOC_IND_KURT']].head(10)

,NEW_DOC_IND_KURT
0,20.000000
1,20.000000
2,0.000000
3,20.000000
4,20.000000
5,20.000000
6,7.037037
7,20.000000
8,20.000000
9,0.000000


In [63]:
df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
df[['NEW_LIVE_IND_SUM']].head(10)

,NEW_LIVE_IND_SUM
0,4
1,4
2,5
3,3
4,3
5,5
6,4
7,4
8,2
9,3


In [64]:
df['NEW_LIVE_IND_STD'] = df[live].std(axis=1)
df[['NEW_LIVE_IND_STD']].head(10)

,NEW_LIVE_IND_STD
0,0.516398
1,0.516398
2,0.408248
3,0.547723
4,0.547723
5,0.408248
6,0.516398
7,0.516398
8,0.516398
9,0.547723


In [65]:
df['NEW_LIVE_IND_KURT'] = df[live].kurtosis(axis=1)
df[['NEW_LIVE_IND_KURT']].head(10)

,NEW_LIVE_IND_KURT
0,-1.875000
1,-1.875000
2,6.000000
3,-3.333333
4,-3.333333
5,6.000000
6,-1.875000
7,-1.875000
8,-1.875000
9,-3.333333


In [66]:
df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
df[['NEW_INC_PER_CHLD']].head(10)

,NEW_INC_PER_CHLD
0,202500.0
1,270000.0
2,67500.0
3,135000.0
4,121500.0
5,99000.0
6,85500.0
7,360000.0
8,112500.0
9,135000.0


In [67]:
df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)
df[['NEW_INC_BY_ORG']].head(20)

,NEW_INC_BY_ORG
0,157500.0
1,135000.0
2,135000.0
3,157500.0
4,162000.0
5,157500.0
6,157500.0
7,157500.0
8,117000.0
9,157500.0


In [69]:
df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['NEW_EMPLOY_TO_BIRTH_RATIO']

0         0.067329
1         0.070862
2         0.011814
3         0.159905
4         0.152418
5         0.093737
6         0.227174
7         0.023820
8              NaN
9         0.139540
10        0.066588
11             NaN
12        0.202173
13        0.214965
14        0.013920
15        0.132562
16        0.101848
17        0.019538
18        0.440456
19        0.179591
20        0.234824
21        0.111509
22        0.386327
23             NaN
24        0.066100
25        0.180718
26        0.140355
27        0.077376
28        0.179708
29        0.097669
            ...   
307481    0.152441
307482    0.016022
307483         NaN
307484    0.025531
307485    0.055959
307486    0.074016
307487         NaN
307488    0.120690
307489    0.017364
307490    0.195260
307491    0.177934
307492    0.077176
307493    0.131523
307494    0.439430
307495    0.355511
307496    0.125569
307497    0.073266
307498    0.193121
307499    0.511773
307500    0.261206
307501    0.080704
307502    0.

In [70]:
df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])
df[['NEW_ANNUITY_TO_INCOME_RATIO']].head(10)

,NEW_ANNUITY_TO_INCOME_RATIO
0,0.121977
1,0.132216
2,0.099999
3,0.219898
4,0.179961
5,0.277952
6,0.241525
7,0.116875
8,0.300677
9,0.149999


In [71]:
df['NEW_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3'] 
df[['NEW_SOURCES_PROD']].head(10)

,NEW_SOURCES_PROD
0,0.003043
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,0.276010
7,NaN
8,0.090840
9,NaN


In [72]:
 df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
 df['NEW_EXT_SOURCES_MEAN'].head(10)

0    0.161787
1    0.466757
2    0.642739
3    0.650442
4    0.322738
5    0.487726
6    0.663607
7    0.627467
8    0.514935
9    0.746644
Name: NEW_EXT_SOURCES_MEAN, dtype: float64

In [73]:
df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
df['NEW_SCORES_STD'].head(10)

0    0.092026
1    0.219895
2    0.122792
3         NaN
4         NaN
5    0.188799
6    0.150717
7    0.122771
8    0.280096
9         NaN
Name: NEW_SCORES_STD, dtype: float64

In [74]:
df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())
df['NEW_SCORES_STD'].head(10)

In [76]:
df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['NEW_CAR_TO_BIRTH_RATIO'].head(10)

0         NaN
1         NaN
2   -0.001365
3         NaN
4         NaN
5         NaN
6   -0.001234
7   -0.000424
8         NaN
9         NaN
Name: NEW_CAR_TO_BIRTH_RATIO, dtype: float64

In [77]:
df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['NEW_CAR_TO_EMPLOY_RATIO'].head(10)

0         NaN
1         NaN
2   -0.115556
3         NaN
4         NaN
5         NaN
6   -0.005431
7   -0.017817
8         NaN
9         NaN
Name: NEW_CAR_TO_EMPLOY_RATIO, dtype: float64

In [78]:
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['NEW_PHONE_TO_BIRTH_RATIO'].head(10)

0    0.119860
1    0.049389
2    0.042791
3    0.032465
4    0.055489
5    0.149696
6    0.113369
7    0.056764
8   -0.000000
9    0.115627
Name: NEW_PHONE_TO_BIRTH_RATIO, dtype: float64

In [79]:
df['NEW_PHONE_TO_EMPLOY_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
df['NEW_PHONE_TO_EMPLOY_RATIO'].head(10)

0    1.780220
1    0.696970
2    3.622222
3    0.203027
4    0.364055
5    1.596977
6    0.499042
7    2.383073
8         NaN
9    0.828628
Name: NEW_PHONE_TO_EMPLOY_RATIO, dtype: float64

In [80]:
df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
df['NEW_CREDIT_TO_INCOME_RATIO'].head(10)

0    2.007889
1    4.790750
2    2.000000
3    2.316167
4    4.222222
5    4.954500
6    9.127053
7    4.250000
8    9.063200
9    3.000000
Name: NEW_CREDIT_TO_INCOME_RATIO, dtype: float64

In [82]:
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

In [86]:
df[bin_feature], uniques

(0         0
 1         1
 2         0
 3         0
 4         0
 5         0
 6         0
 7         0
 8         0
 9         0
 10        0
 11        0
 12        0
 13        1
 14        0
 15        0
 16        1
 17        0
 18        0
 19        0
 20        0
 21        0
 22        1
 23        0
 24        1
 25        0
 26        0
 27        0
 28        0
 29        0
          ..
 307481    0
 307482    0
 307483    0
 307484    1
 307485    0
 307486    0
 307487    0
 307488    1
 307489    0
 307490    0
 307491    0
 307492    0
 307493    0
 307494    1
 307495    1
 307496    1
 307497    0
 307498    0
 307499    0
 307500    0
 307501    0
 307502    0
 307503    0
 307504    0
 307505    0
 307506    1
 307507    0
 307508    0
 307509    0
 307510    1
 Name: FLAG_OWN_REALTY, Length: 307507, dtype: int64,
 Index(['Y', 'N'], dtype='object'))

In [88]:
# Categorical features with One-Hot encode
df, cat_cols = one_hot_encoder(df, False)

In [91]:
len(cat_cols)

133